In [2]:
import numpy as np
import torch
import torch.nn as nn


In [3]:
def softmax(Z):
    Z = np.exp(Z - Z.max(axis =1, keepdims = True))
    return Z/ Z.sum(axis = 1, keepdims = True)



In [4]:
T,d = 7, 50
attn = nn.MultiheadAttention(d,10, bias=False, batch_first=True)
M  = torch.triu(-float("inf")*torch.ones(T,T),1)
X = torch.randn(500, T, d)
Y_, A_ = attn(X,X,X, attn_mask = M)


In [5]:
Y_.shape

torch.Size([500, 7, 50])

In [6]:
import math
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.0, max_len=16):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1), :].squeeze(1)
        return x

In [7]:
pos = PositionalEncoding(d_model=50, max_len=50)
encoder_layers = nn.TransformerEncoderLayer(d_model=50, nhead=2, dim_feedforward=100, dropout=0.1)
cell = nn.TransformerEncoder(encoder_layers, num_layers=10)

In [8]:
out = pos(X)
out = cell(out)

In [10]:
X.shape

torch.Size([500, 7, 50])